# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
file_to_load = "WeatherData.csv"

# Read Purchasing File and store into Pandas data frame
vacation_df = pd.read_csv(file_to_load)
vacation_df.dropna()
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,0,Okha,37,RU,1579548416,96,53.57,142.95,-6.93,9.40
1,1,Hobart,75,AU,1579548416,71,-42.88,147.33,57.20,10.29
2,2,Bredasdorp,99,ZA,1579548375,88,-34.53,20.04,64.99,16.11
3,3,Henties Bay,0,NaN,1579548417,86,-22.12,14.28,66.99,7.00
4,4,Mataura,94,NZ,1579548417,96,-46.19,168.86,57.00,1.99
...,...,...,...,...,...,...,...,...,...,...
543,543,Praia,0,CV,1579548471,47,14.92,-23.51,77.00,14.99
544,544,Barabai,84,ID,1579548476,94,-2.58,115.38,73.89,1.72
545,545,Hede,66,CN,1579548476,69,33.77,120.25,43.03,11.18
546,546,Nanakuli,75,US,1579548476,94,21.39,-158.15,75.20,10.49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = vacation_df["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
criteria_df = vacation_df[vacation_df['Max_Temp'].between(70, 80)]
criteria_df = criteria_df[criteria_df['Wind_Speed']<10]
criteria_df = criteria_df = criteria_df[criteria_df['Cloudiness'] == 0]
criteria_df = criteria_df[criteria_df['Max_Temp']<78.80]
criteria_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
139,139,Atar,0,MR,1579548432,10,20.52,-13.05,78.03,9.89
169,169,Nelson Bay,0,AU,1579548435,88,-32.72,152.15,71.01,4.70
380,380,Port Macquarie,0,AU,1579548460,100,-31.43,152.92,70.00,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [14]:
hotel_df = criteria_df
                                 
hotel_df['Hotel Name'] = ""

hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
139,139,Atar,0,MR,1579548432,10,20.52,-13.05,78.03,9.89,
169,169,Nelson Bay,0,AU,1579548435,88,-32.72,152.15,71.01,4.70,
380,380,Port Macquarie,0,AU,1579548460,100,-31.43,152.92,70.00,3.36,


In [15]:
# params dictionary to update each iteration
params = {
    "radius": 7000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to identify Hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [16]:
narrowed_city_df = hotel_df
narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name,Hotel_Name
139,139,Atar,0,MR,1579548432,10,20.52,-13.05,78.03,9.89,,Auberge mer et desert
169,169,Nelson Bay,0,AU,1579548435,88,-32.72,152.15,71.01,4.70,,Ramada Resort by Wyndham Shoal Bay
380,380,Port Macquarie,0,AU,1579548460,100,-31.43,152.92,70.00,3.36,,Port Macquarie Hotel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_locations = narrowed_city_df[['Lat', 'Lng']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))